In [14]:
import os
import json
from pyannote.core import Annotation
from mexca.text.transcription import AudioTextIntegrator, AudioTranscriber
from mexca.audio.speaker_id import SpeakerIdentifier


In [3]:
with open(os.path.join(
            'tests', 'reference_files', 'transcription_dutch_1_second.json'), 'r') as file:
        transcription = json.loads(file.read())

In [4]:
with open(os.path.join(
            'tests', 'reference_files', 'reference_audio_5_seconds.json'), 'r') as file:
        ref_speakers = Annotation.from_json(json.loads(file.read()))

In [5]:
def segment_text(transcription, ref_speakers):
    text, char_starts, char_ends = transcription['transcription'], transcription['start_timestamps'], transcription['end_timestamps']
    
    output = {'speech_start':[],'speech_end':[],'speaker':[],'text':[]}
    for speech_turn, _, speaker in ref_speakers.itertracks(yield_label=True):
        speech_start, speech_end = 1000 * round(speech_turn.start,3), 1000 * round(speech_turn.end,3) #convert to ms
        extracted_text = ''.join([char  for i,char in enumerate(text) if (char_ends[i]>=speech_start and char_starts[i]<=speech_end)])
        output['speech_start'].append(speech_start)
        output['speech_end'].append(speech_end)
        output['speaker'].append(speaker)
        output['text'].append(extracted_text)

    return output



In [7]:
out  = segment_text(transcription,ref_speakers)

{'speech_start': [260.0],
 'speech_end': [950.0],
 'speaker': ['SPEAKER_00'],
 'text': [' en groen als']}

In [42]:
out

{'speech_start': [260.0],
 'speech_end': [950.0],
 'speaker': ['SPEAKER_00'],
 'text': [' en groen als']}

In [8]:
with open("tests/reference_files/text_audio_integration.json", "w") as write_file:
    json.dump(out, write_file, indent=4)

    

In [18]:
with open(os.path.join('tests', 'reference_files', 'text_audio_integration.json'), 'r') as file:
    predicted_output = json.loads(file.read())

In [13]:
model = AudioTextIntegrator(language='dutch')

06/27/2022 16:08:02 - INFO - huggingsound.speech_recognition.model - Loading model...


In [3]:
audio_filepath = os.path.join('tests', 'audio_files', 'test_dutch_5_seconds.wav')

In [4]:
output_predicted = model.apply(audio_filepath)

100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


{'transcription': 'maak en groen als', 'start_timestamps': [60, 100, 180, 220, 260, 300, 380, 440, 480, 540, 580, 620, 680, 740, 800, 840, 880], 'end_timestamps': [80, 120, 200, 240, 280, 320, 400, 460, 500, 560, 600, 640, 700, 760, 820, 860, 900], 'probabilities': [0.839084267616272, 0.9999984502792358, 0.9999985694885254, 0.9999988079071045, 0.9404094815254211, 0.9997819066047668, 0.598744809627533, 0.9999990463256836, 0.9999988079071045, 0.9999977350234985, 0.9999982118606567, 0.9999430179595947, 0.999998927116394, 0.9743364453315735, 0.9999988079071045, 0.9999505281448364, 0.999866247177124]}
[ 00:00:00.257 -->  00:00:00.949] A SPEAKER_00


TypeError: _segment_text() takes 2 positional arguments but 3 were given

In [7]:
audio_model = SpeakerIdentifier()

In [15]:
text_model = AudioTranscriber(language='dutch')

06/27/2022 16:08:56 - INFO - huggingsound.speech_recognition.model - Loading model...


In [9]:
audio_output = audio_model.apply(audio_filepath)

In [17]:
text= output = text_model.apply(audio_filepath)

100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


In [12]:
len(audio_output)
len(output_predicted)


NameError: name 'output_predicted' is not defined